# Names (Enter your names below)
**Your Name and JHED:** ...

**Partner's Name and JHED (If applicable):**  ...

# Lab 1: Heart Rate Estimation and Signal De-noising
By **Benjamín Béjar Haro** and edited by **Kwame Kutten**

In this lab we will address the problem of heart rate estimation from noisy observations. For a healthy individual, and under ideal conditions, this task might be easy to accomplish. However, different disorders together with different sources of error cause distortions to the signal rendering the estimation task more difficult. We will study how prior information about the signal and the interference can be exploited in order to improve the estimation task under less ideal conditions.

We will start detecting the QRS-complex assuming we have a template of the underlying waveform. Later, we will move to a more realistic scenario where we will be implementing a simplified version of the Pan-Tompkins algorithm [Pan-Tompkins algorithm](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=4122029) and we will test it on real ECG data recordings.

### Objectives
By the end of this session you should be able to understand the main challenges in the estimation of EGC signals. How to work from modeling assupmtions in order to exploit your prior knowledge in the estimation task.

***

### Task 1. Signal generation and template matching
In this task we will implement a simple version of a QRS-complex detector by using template matching, thresholding and non-maximum suppresion on the
observed ECG signal. We will then use those detections to provide an estimate of the heart rate.

* **Task 1A: Signal Generation [15 points total] -** We are going to create a synthetic EGC signal that we will be using to test our detector algorithm. Each pulse should have a sampling rate of $f_s = 256\ Hz$. The goal of this task is to create a basic model for a ECG signal. You can think of the signal as the convolution of a canonical pulse shape $\varphi(t)$ with a stream of Dirac delta functions: 

   $$x(t) = \sum_{k=1}^K a_k\, \varphi(t-t_k) = \varphi(t) \ast \sum_{k=1}^K a_k\, \delta(t-t_k),$$ 

   where $K$ is the number of pulses observed, $a_k$ represent the amplitudes of the different pulses and $t_k$ correspond to the locations of the pulses in time. 
  * Generate a stream of $K = 10$ equally spaced pulses over a time span of $10\ s$ using the canonical pulse shape generated by the `ecg_wave()` function defined in the notebook and constant unit amplitudes $a_k=1$ [7 points]. 
  * Plot the generated signal over time [6 points]
  * Add x-axis and y-axis labels [2 points]


In [ ]:
# ============================================================================
# import modules
# ============================================================================
# Note that this part of the code needs to be run prior to any other code cell

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal

# inline plots
%matplotlib inline
sns.set()

# ============================================================================
# functions
# ============================================================================
def ecg_wave(x):
    """
    This function generates a single synthetic ECG template of unit length (support).
    For example, for a sampling frequency of f_s, use x = np.linspace(0,1, f_s, endpoint=False)
    """
    
    # compute signal (superposition of splines)
    return 0.3 * signal.bspline(3*3*x-7.5,2)\
         + 0.15* signal.bspline(3*4*x-2,3)\
         + signal.bspline(3*2*2*x-6,1) - 0.2 * signal.bspline(3*4*x-5,1) - 0.4 * signal.bspline(3*4*x-7,1)


# ============================================================================
# Write Your code here
# ============================================================================


* **Task 1B: Simulate Noise [5 points total]**
  * Generate a noisy version of the synthetic ECG signal generated by adding Gaussian noise with standard deviation $\sigma=0.5$ [2 points]. 
  * Plot the noisy observations [3 points].

In [ ]:
# ============================================================================
# Write Your code here
# ============================================================================


* **Task 1C: Find Peaks [10 points total] -** Implement a QRS-complex detector on the noisy signal you just generated. For that purpose we will use template matching, thresholding, and non-maximum suppression.
  * First you should implement template matching by correlating the noisy signal with a template ECG pulse using `np.correlate` [2 points]
  * Next normalize the signal to range $[0,1]$ using `noralize_range` [2 points]
  * Then plot the result [2 points]
  * Select a threshold value in $[0,1]$ that will separate QRS peeks from other peaks [1 point]
  * Next use `signal.find_peaks` with the `height` parameter set to the your chosen threshold to find locations of the QRS peaks [2 points]
  * Print the times of each peak in seconds [1 points]

In [ ]:
def normalize_range(x):
    """
    Normalizes the signal such that its minimum value is 0 and its maximum value is 1,
    provided the signal is not identically zero.
    """
    # check that there are non-zero elements
    if np.any(x):
        
        # subtract minimum
        minx = np.min(x)
        z = x - minx
        
        # divide by max value
        maxz = np.max(z)
        return z/maxz
    
    else:    
        return x
    
# ============================================================================
# Write Your code here
# ============================================================================


* **Task 1D: RR Intervals [10 points total]** 
  * Verify your results by ploting the original (synthetic) ECG signal overlaid with markers of peek locations. Keep in mind to compensate for any delay you might have introduced by filtering. [5 points]
  * Find the $R$ to $R$ interval sequence $r_n$ and its average value as $$ \bar R = \frac{1}{N}\sum_{n=0}^N r_n, $$ where $N$ is the number intervals between peaks.  Use this to display an estimate of the average heart beat rate in beats per minute (bpm) [5 points].


In [ ]:
# ============================================================================
# Write your code here
# ============================================================================


### Task 2. QRS complex detection via Pan-Tompkins algorithm
Now let us consider a more realistic scenario where we have a noisy ECG signal and the waveform of the QRS-complex is unknown. There are different sources of noise that can be present during an ECG acquisition. In addition to high-frequency thermal noise, it is also common to observe the presence of low-frequency interference coming from breathing. 
When it comes to pathologies, different non-additive distortions might be present on the ECG signal 
that alter the shape of the QRS-complex itself but for the purpose of this task we just assume a healthy  individual where signal distortion comes solely from the acquisition process. The procedure that we will employ in order to estimate the locations of the QRS-complex is based on the Pan-Tompkins algorithm [Pan-Tompkins algorithm](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=4122029).

**Important Note 1:** You may need to **adjust the `myPath` variable below.**

**Important Note 2:** If you are using **Google Colab you must follow these steps**

1. Upload **ecg_mitnst.json** into your Google Drive.  If you placed it in a sub folder called **myFolder** the use `myPath = "myFolder/ecg_mitnst.json"`. Otherwise use `myPath = "ecg_mitnst.json"`
2. Run the folloing cell and follow displayed instructions

In [ ]:
# ============================================================================
# Edit myPath variable below if necessary
# ============================================================================
import json

def read_data(path):
    try:
        # Executes if running in Google Colab
        from google.colab import drive
        drive.mount('gdrive/')
        path = 'gdrive/My Drive/'+path
    except:
        # Executes if running locally (e.g. Anaconda)
        pass

    with open(path,'r') as infile: data = json.load(infile)
    return data

# print data to see dictionary fields
myPath = "ecg_mitnst.json" # Change myPath to location of data if necessary
data = read_data(myPath)
print(data.keys())

**Task 2A: Effect of noise [10 points]:** The `data` dictionary contains four fields.  `fs` is the sampling frequency, `voltage` is the measured voltage in *mV* at corresponding `time` points and `t_k` contains the indices of the peaks. The signal corresponds to a sample ECG from the [MIT noise stress dataset](https://physionet.org/physiobank/database/nstdb/)
that has been downloaded from [physionet.org](https://physionet.org/physiobank/database/).
Note the presence of a strong low-frequency component.
  * Plot the voltage over time [5 points]
  * Add markers to indicate the given peaks [5 points]

In [ ]:
# ============================================================================
# Write your code here
# ============================================================================


#### **Pan-Tompkins Algorithm**
In order to deal with noise we will implement a simplified version of the Pan-Tompkins algorithm. For your convinence we have provided `boxFilter` and `triangleFilter` functions. Proceed as follows:

**Task 2B [10 points total]:** _Bandpass filtering:_ In this first step we want to eliminate as much as possible interference that is present in our acquired signal. In order to do that we can filter the signal and only leave the frequency range
that is of interest for our task (e.g. $5-15$ Hz as suggested in the original Pan-Tompkins paper) the bandpass filtering operation as the concatenation of a lowpass and a highpass filtering steps:

* _Highpass filtering:_ A highpass filter can be thought of as removing the lower frequency component of the signal. That means that we could use a lowpass filter to get a lowpass version of the original signal and then subtract the result from the original series. 
   * We will use a lowpass filter of triangular shape and length $L$. Determine the length of the filter based on the required passband (i.e. the filter should attenuate significantly beyond $5$ Hz) or empirically by adjusting the value of $L$. Plot the original signal together with the lowpass filtered version. The latter one should describe the trend in the ECG recording [4 points]. 
   * Then plot the signal with the lowpass version subtracted to leave high-pass filtered signal [4 points].

* _Lowpass filtering:_ Use an averaging (box) filter to reduce high-frequency noise. Filter the signal with a filter of length $L=10$ taps [3 points].



In [ ]:
def boxFilter(L):
    """
    Genearte box filter of length L
    """
    h = np.ones(L)
    return h/np.sum(h)

def triangleFilter(L):
    """
    Generate triangle filter of length L
    """
    h = np.convolve(boxFilter(L//2), boxFilter(L//2))
    h = np.concatenate(([0],h))
    if L % 2: h = np.concatenate((h,[0]))
    return h/np.sum(h)

# ============================================================================
# Write your code here
# ============================================================================


**Task 2C [10 points total]:** _Differentiation_
 * Use a 1D numpy array to define the finite difference filter [3 points] $$d_n = \frac{1}{8}\big(\delta_{n-2} + 2\delta_{n-1} -2\delta_{n+1} - \delta_{n+2}\big)$$
 * Then differentiate the resulting signal in order to localize the region of steepest  slope  in  the  QRS-complex [2 points].  
 * Plot the result [5 points]

In [ ]:
# ============================================================================
# Write your code here
# ============================================================================


**Task 2D [10 points total]:** _Signal squaring_
 * Square the signal after differentiation [5 points]
 * Plot the result [5 points]

In [ ]:
# ============================================================================
# Write your code here
# ============================================================================


**Task 2E [10 points total]:** _Integration_ 
 * Integrate the resulting signal from the squaring operation with a box window of length $L=50$ [5 points]
 * Plot the resulting signal [5 points]

In [ ]:
# ============================================================================
# Write your code here
# ============================================================================


**Task 2F [10 points total]:** _Peak detection_ 
 * Use `signal.find_peaks` with an appropriate `height` threshold to estimate locations of QRS peaks [2 points]
 * Replot the previous plot with markers at these peak locations [4 points].
 * In a differnt plot display the heart beat rate over time estimated from the **R** to **R** intervals [4 points]

In [ ]:
# ============================================================================
# Write your code here
# ============================================================================
